In [81]:
import pandas as pd
import altair as alt
pd.set_option('display.max_columns', None)


In [83]:
df = pd.read_csv('electric_vehicles_spec_2025.csv')
df.head()

,brand,model,top_speed_kmh,battery_capacity_kWh,battery_type,number_of_cells,torque_nm,efficiency_wh_per_km,range_km,acceleration_0_100_s,fast_charging_power_kw_dc,fast_charge_port,towing_capacity_kg,cargo_volume_l,seats,drivetrain,segment,length_mm,width_mm,height_mm,car_body_type,source_url
0,Abarth,500e Convertible,155,37.8,Lithium-ion,192.0,235.0,156,225,7.0,67.0,CCS,0.0,185,4,FWD,B - Compact,3673,1683,1518,Hatchback,https://ev-database.org/car/1904/Abarth-500e-C...
1,Abarth,500e Hatchback,155,37.8,Lithium-ion,192.0,235.0,149,225,7.0,67.0,CCS,0.0,185,4,FWD,B - Compact,3673,1683,1518,Hatchback,https://ev-database.org/car/1903/Abarth-500e-H...
2,Abarth,600e Scorpionissima,200,50.8,Lithium-ion,102.0,345.0,158,280,5.9,79.0,CCS,0.0,360,5,FWD,JB - Compact,4187,1779,1557,SUV,https://ev-database.org/car/3057/Abarth-600e-S...
3,Abarth,600e Turismo,200,50.8,Lithium-ion,102.0,345.0,158,280,6.2,79.0,CCS,0.0,360,5,FWD,JB - Compact,4187,1779,1557,SUV,https://ev-database.org/car/3056/Abarth-600e-T...
4,Aiways,U5,150,60.0,Lithium-ion,NaN,310.0,156,315,7.5,78.0,CCS,NaN,496,5,FWD,JC - Medium,4680,1865,1700,SUV,https://ev-database.org/car/1678/Aiways-U5


The Electric Vehicle Specifications Dataset (2025) is collected from kaggle(https://www.kaggle.com/datasets/urvishahir/electric-vehicle-specifications-dataset-2025). This dataset provides detailed information on electric cars from over 59 global manufacturers, including performance, battery, drivetrain, and design attributes. It contains data such as range, efficiency, top speed, acceleration, battery capacity, charging power, and vehicle dimensions across various body types and market segments.

### How do electric vehicles from different brands and body types compare in terms of driving range and energy efficiency?

In [77]:
import altair as alt
import pandas as pd

for c in ['brand','car_body_type','drivetrain']:
    if c in df.columns:
        df[c] = df[c].fillna('Unknown')

brands      = sorted(df['brand'].unique().tolist())
bodies      = sorted(df['car_body_type'].unique().tolist())
selectEV = alt.selection_single(
    name='Select',
    fields=['brand', 'car_body_type'],
    bind={
        'brand':        alt.binding_select(options=brands,      name='Brand: '),
        'car_body_type':alt.binding_select(options=bodies,      name='Body: ')
    }
)

chart = (
    alt.Chart(df)
      .mark_circle(size=40)
      .add_selection(selectEV)
      .encode(
          x=alt.X('range_km:Q', title='Range (km)'),
          y=alt.Y('efficiency_wh_per_km:Q', title='Efficiency (Wh/km)'),
          color=alt.Color('car_body_type:N', title='Body'),
          tooltip=['brand:N','model:N','car_body_type:N',
                   'range_km:Q','efficiency_wh_per_km:Q','battery_capacity_kWh:Q'],
          # highlight when the current row matches all chosen field values
          opacity=alt.condition(selectEV, alt.value(0.85), alt.value(0.12))
      )
      .properties(width=650, height=380, title='EVs: Range vs Efficiency')
)

chart


/opt/anaconda3/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype

alt.Chart(...)

This interactive plot helps to dynamically explore and compare electric vehicle performance across different brands and body types, making complex multidimensional data intuitive to interpret. By binding dropdown menus to brand and body type, users can focus on specific segments of interest without being overwhelmed by the full dataset. The scatterplot format effectively visualizes the relationship between driving range and energy efficiency, while interactivity enhances engagement and supports pattern discovery such as identifying the range vs efficient for each body type and how it varies across several brands.

### How do efficiency, range, performance, and battery capacity vary across different electric vehicles (EVs)?

In [89]:
import altair as alt
import pandas as pd

for c in ['brand','car_body_type','drivetrain']:
    if c in df.columns:
        df[c] = df[c].fillna('Unknown')

alt.data_transformers.disable_max_rows()

brush = alt.selection_interval()

left = (
    alt.Chart(df)
      .mark_circle(size=70, stroke='black', strokeWidth=0.4)
      .encode(
          x=alt.X('range_km:Q', title='Range (km)'),
          y=alt.Y('efficiency_wh_per_km:Q', title='Efficiency (Wh/km)'),
          color=alt.condition(brush, 'car_body_type:N', alt.value('#d3d3d3')),
          tooltip=[
              'brand:N','model:N','car_body_type:N','drivetrain:N',
              'range_km:Q','efficiency_wh_per_km:Q','battery_capacity_kWh:Q'
          ]
      )
      .add_params(brush)
      .properties(width=420, height=320, title='Range vs Efficiency')
)

right = (
    alt.Chart(df)
      .mark_circle(size=70, stroke='black', strokeWidth=0.4)
      .encode(
          x=alt.X('top_speed_kmh:Q', title='Top speed (km/h)'),
          y=alt.Y('acceleration_0_100_s:Q', title='0–100 km/h (s)'),
          color='car_body_type:N',
          tooltip=[
              'brand:N','model:N','car_body_type:N','drivetrain:N',
              'top_speed_kmh:Q','acceleration_0_100_s:Q'
          ]
      )
      .transform_filter(brush)
      .properties(width=420, height=320, title='Top speed vs 0–100')
)
bar = (
   alt.Chart(df).mark_circle(size=80).encode(
    x='battery_capacity_kWh:Q',
    y='range_km:Q',
    color='car_body_type:N',
    tooltip=['brand','model','battery_capacity_kWh','range_km']
).transform_filter(brush)

).properties(width = 420, height = 320, title = 'Range vs Battery Capacity')

# Layout
((left | right) & bar).configure_legend(title=None)


/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/c

alt.VConcatChart(...)

This interactive brushing and linking visualization helps to explore  multidimensional relationships among EV specifications. The Range vs Efficiency scatter plot enables users to visually identify trade-offs between driving distance and energy consumption across various body types. By brushing a subset of vehicles, users can instantly see how those same models perform in terms of Top Speed vs Acceleration, revealing links between efficiency and performance. The third scatter plot, Battery Capacity vs Range, shows how effectively different EVs convert stored energy into distance traveled. Together, these linked views make it easy to compare brands and body types, highlight outliers, and uncover patterns that would be difficult to detect in static charts.

### For a particular car brand, how many vehicles of each body type have been launched, and how do their battery capacity and driving range compare across those models?

In [95]:
import altair as alt

for c in ['brand','car_body_type']:
    if c in df.columns:
        df[c] = df[c].fillna('Unknown')

alt.data_transformers.disable_max_rows()

sel_brand = alt.selection_point(fields=['brand'], on='click', clear='dblclick')
sel_body  = alt.selection_point(fields=['car_body_type'], on='click', clear='dblclick')
sel_brush = alt.selection_interval()  # drag on scatter to filter numeric ranges

combined = sel_brand & sel_body & sel_brush

bar_brand = (
    alt.Chart(df)
      .mark_bar()
      .encode(
          y=alt.Y('brand:N', sort='-x', title='Brand'),
          x=alt.X('count():Q', title='Vehicles'),
          color=alt.condition(sel_brand, 'brand:N', alt.value('#dddddd'))
      )
      .add_params(sel_brand)
      .transform_filter(sel_body)
      .transform_filter(sel_brush)
      .properties(width=300, height=600, title='Total no.of vehicles launched in each brand')
)

bar_body = (
    alt.Chart(df)
      .mark_bar()
      .encode(
          y=alt.Y('car_body_type:N', sort='-x', title='Body Type'),
          x=alt.X('count():Q', title='Vehicles'),
          color=alt.condition(sel_body, 'car_body_type:N', alt.value('#dddddd'))
      )
      .add_params(sel_body)
      .transform_filter(sel_brand)
      .transform_filter(sel_brush)
      .properties(width=400, height=180, title='Total no.of vehicles for each car type')
)

scatter = (
    alt.Chart(df)
      .mark_circle(size=70, stroke='black', strokeWidth=0.4)
      .encode(
          x=alt.X('range_km:Q', title='Range (km)'),
          y=alt.Y('battery_capacity_kWh:Q', title='Battery Capacity(kWh)'),
          color='car_body_type:N',
          tooltip=['brand:N','model:N','car_body_type:N',
                   'range_km:Q','battery_capacity_kWh:Q',
                   'top_speed_kmh:Q','acceleration_0_100_s:Q']
      )
      .add_params(sel_brush)
      .transform_filter(sel_brand)
      .transform_filter(sel_body)
      .properties(width=400, height=320, title='Range vs Battery Capacity')
)

# --- layout (cross-filtering) ---
(bar_brand) | (bar_body & scatter)


/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/c

alt.HConcatChart(...)

This interactive cross-filtering plot is suitable because it connects categorical insights with quantitative performance measures in a single view. By clicking on a specific brand in the first bar chart, users can immediately see how that brand’s vehicles are distributed across different body types, while the linked scatter plot reveals how their battery capacities relate to range. The coordinated interaction between the bar charts and scatter plot allows users to filter, highlight, and compare subsets of data dynamically, making it easy to explore whether certain body styles within a brand achieve better range efficiency or rely on larger batteries. This combination provides a clear, interactive overview of each manufacturer’s EV lineup and performance diversity.